In [21]:
import pandas as pd
import os
import re
from datetime import datetime


#### Reformat data into old Fitbit export style.


In [22]:
filepath = r'~/Downloads/takeout-20231226T235127Z-001/Takeout/Fitbit/Global Export Data'
json_files = os.listdir(os.path.expanduser(filepath))
filtered_files = [file for file in json_files if re.match(r'sleep-\d{4}-\d{2}-\d{2}\.json$', file)]
filtered_files.sort()
filtered_files[:5] # Check first 5 are as expected.

['sleep-2016-08-29.json',
 'sleep-2016-09-28.json',
 'sleep-2016-10-28.json',
 'sleep-2016-11-27.json',
 'sleep-2016-12-27.json']

In [23]:
dfs = []
for file in filtered_files:
    file_path = os.path.join(r'~/Downloads/takeout-20231226T235127Z-001/Takeout/Fitbit/Global Export Data/', file)
    df = pd.read_json(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [24]:
unnested = pd.json_normalize(df['levels'])

In [25]:
df = pd.merge(
    left=df[['startTime', 'endTime', 'minutesAsleep', 'minutesAwake', 'timeInBed']], 
    right=unnested.iloc[:, 2:], 
    left_index=True, 
    right_index=True
)

In [26]:
df.rename(columns={
    'summary.wake.count':'Number of Awakenings',
    'summary.rem.minutes': 'Minutes REM Sleep',
    'summary.light.minutes':'Minutes Light Sleep',
    'summary.deep.minutes':'Minutes Deep Sleep',
}, inplace=True)

In [27]:
tmp_col = df['Number of Awakenings'].copy()
df.drop(columns=['Number of Awakenings'], inplace=True)
df.insert(4, 'Number of Awakenings', tmp_col)

In [28]:
df = df[[
    'startTime', 
    'endTime', 
    'minutesAsleep', 
    'minutesAwake', 
    'Number of Awakenings', 
    'timeInBed', 
    'Minutes REM Sleep',
    'Minutes Light Sleep',
    'Minutes Deep Sleep',
]]

In [29]:
df = df.astype({'startTime':'datetime64[ns]', 'endTime':'datetime64[ns]'})

In [30]:
def convert_to_fitbit_time(cols):
    """Converts millisecond UTC timestamp into native 12h time."""
    for col in cols:
        df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d %I:%M%p'))
        df[col] = df[col].apply(lambda x: x.replace(' 0', ' '))

        
convert_to_fitbit_time(['startTime', 'endTime'])

In [31]:
df.head()

,startTime,endTime,minutesAsleep,minutesAwake,Number of Awakenings,timeInBed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,2016-09-27 10:21PM,2016-09-28 7:33AM,521,26,NaN,551,NaN,NaN,NaN
1,2016-09-26 11:54PM,2016-09-27 7:34AM,439,21,NaN,460,NaN,NaN,NaN
2,2016-09-26 12:14AM,2016-09-26 7:24AM,403,27,NaN,430,NaN,NaN,NaN
3,2016-09-24 10:41PM,2016-09-25 9:28AM,599,47,NaN,647,NaN,NaN,NaN
4,2016-09-23 11:07PM,2016-09-24 5:22AM,357,17,NaN,374,NaN,NaN,NaN


In [32]:
savepath = r'~/OneDrive/Documents/531_data' # Make this the location where you want to save the new DataFrame.
df.to_csv(f'{savepath}/fitbit_sleep_export.csv', index=False, header=False, na_rep='N/A')